**Project:** Protein Function Improvement through Protein Language Model Embeddings using Machine Learning Approach

**Goal:**
The goal of this project is to predict fluorescence intensity from amino acid sequences. Here, the protein sequence is represented in the form embeddings (mean embeddings of residues in the sequence) extracted from Protein Language Model (PLM). The project involves training various models and selecting the most optimal one for further prediction. A similar workflow can be beneficial for predicting other protein function properties.

**Embeddings from Protein Language Models:** These models are pre-trained with a large protein sequence database to understand the sequences and structures of proteins, mapping them into dense vector representations, or embeddings. Embeddings capture complex relationships between amino acids and can be used as features for downstream tasks like protein function prediction.

**Steps:**
1. Data Collection
2. Feature Extraction
3. Preprocessing and Normalization
4. Model Selection and Training
5. Model Evaluation and Hyperparameter Tuning
6. Model Prediction

**Step 1: Data Collection**
<br>Dataset: Deep Mutational Scanning (DMS) data for the parent green fluorescent protein (GFP) protein and its mutants, originally derived from Sarkisyan et al. (2016). Pre-computed data splits from Rao et al. (2020) will be used. The pre-computed data splits contain 'train', 'valid', and 'test' sets. Evaluation will be made using the 'test' set across all approaches. For testing purposes, only 10% of the 'train' set will be retained.  

`Protein sequence`: GFP variants<br>
`Target value`: Fluorescence intensity<br>

**File:** fluorescence.csv (use scripts/create_dataset.py to create dataset for use)
<br>`wget http://s3.amazonaws.com/songlabdata/proteindata/data_raw_pytorch/fluorescence.tar.gz`
<br>`tar -xzvf fluorescence.tar.gz`
<br>`python scripts/create_dataset.py -d fluorescence -o fluorescence.csv -t 0.1 (use -t 1.0 to use full dataset)`
<br>`rm -rf fluorescence`
<br>`rm fluorescence.tar.gz`

**Prerequisites:**
JupyterLab is an open-source web-based interactive development environment (IDE) primarily used for working with Jupyter notebooks, code, and data (advanced version of Jupyter Notebook). It is generally a good idea to install JupyterLab in a dedicated environment to avoid conflicts with other packages.

To create a new conda environment, run:<br>
	&emsp;&emsp;&emsp;`conda create -n mlearn python=3.12`<br>
	&emsp;&emsp;&emsp;`conda activate mlearn`

*(alternatively, create conda environment from the provided yml file: `conda env create -f mlearn.yml`)*<br>

To install jupyer-lab, run:<br>
    &emsp;&emsp;&emsp;`conda install -c conda-forge jupyterlab`

Once JupyterLab is installed, you can start it by running:<br>
    &emsp;&emsp;&emsp;`jupyter-lab`

This will launch JupyterLab in your default web browser. It will typically open at `http://localhost:8888` (or another port if 8888 is already in use).

In [ ]:
# install required packages
!pip install pandas scipy scikit-learn ipywidgets optuna-integration hyperopt scikit-optimize torch transformers

!pip install lazypredict
#!pip install autogluon
#!pip install mljar-supervised
#!pip install pycaret
# note: compatibility issue might arise

In [2]:
# try to install scikit-learn-intelex for speed
!pip install scikit-learn-intelex==2023.1.1

ERROR: Could not find a version that satisfies the requirement scikit-learn-intelex==2023.1.1 (from versions: none)
ERROR: No matching distribution found for scikit-learn-intelex==2023.1.1


In [ ]:
# When running in Google Colab
# (i) Create a copy of data and notebook in your own drive
# Mount Drive
#from google.colab import drive
#drive.mount('/content/drive')

**Step 2: Data Preparation and Processing**
<br>Use precomputed data splits (in this study) or perform data splits for training using train_test_split from sklearn.

In [5]:
# Define arguments
seqcol = 'sequence' # column to sequence
ycol = 'label' # column to score/label
seed = 42 # random seed to use
test_size = 0.2 # test size for data splits
checkpoint="facebook/esm2_t6_8M_UR50D" # model to use

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# (A) In the case where data split is pre-computed
# Load training data
# csv_file="/content/drive/MyDrive/Colab Notebooks/fluorescence.csv"
csv_file = "../data/fluorescence.csv"
df = pd.read_csv(csv_file)
print(df.head(2))
print(len(df), len(df[df['split']=='train']), len(df[df['split']=='valid']), len(df[df['split']=='test']))

# (B) In the case where the data split is not defined
def get_data_split(df, test_size):
    train, test = train_test_split(
        df,
        test_size=test_size,
        random_state=seed,
    )
    train['split'] = 'train'
    test['split'] = 'test'
    df = pd.concat([train,test])
    return df

                                            sequence     label  split
0  SKGEELFTGVVPILVELDGEVNGHKFSVSGEGEGDATYGKLTLKFI...  3.586350  train
1  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...  3.612519  train
5173 2097 524 2552


**Step 3: Embedding Extraction from Protein Language Model**
<br>Protein language models (pLMs) generate high-dimensional embeddings of a protein sequence on a per-residue level. The embeddings capture complex relationships between amino acids and can be used as features for downstream tasks like binding affinity prediction. For per-protein level prediction, the mean residue embeddings are used. As an example, the smallest ESM2 model variant will be used, i.e. esm2_t6_8M_UR50D.

The final embedding used for the downstream task often requires pooling, a method that concatenates the per-residue-level embeddings to form a fixed-length vectors of protein-level representation. Pooling choice could affect the downstream prediction tasks since the method determines what information is preserved or discarded from the sequence. Mean pooling captures the global, average signal across all residues, max pooling emphasizes the presence of critical residues or motifs, and sum-pooling aggregates residue-level information while preserving overall signal magnitude. Here, we evaluate the performance of mean, max and sum-pooled embeddings.


In [6]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

# set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda': torch.cuda.empty_cache()
print('device:',device)

# define a function to compute embedding (define the column to extract sequence)
def compute_embeddings(df, seq_col, checkpoint, pool):
    # load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModel.from_pretrained(checkpoint, dtype=torch.float16)
    model = model.to(device)
    if device.type == 'cuda':
        model = model.half()
        
    all_embeddings = []
    for i in range(0,len(df[seq_col])):
        inputs = tokenizer(df[seq_col].loc[i], return_tensors="pt", max_length = 1024, truncation=True, padding=True).to(device)
        with torch.no_grad():
            # get pooled embeddings
            outputs = model(**inputs).last_hidden_state.cpu()
            if pool == 'mean':
                embedding = torch.mean(outputs, dim=1)
            elif pool == 'sum':
                embedding = torch.sum(outputs, dim=1)
            elif pool == 'max':
                embedding, _ = torch.max(outputs, dim=1)
            else:
                raise ValueError(f"Unsupported pooling method: {pool} (mean, max, sum)")
            all_embeddings.append( embedding )
    all_embeddings = torch.cat(all_embeddings, dim=0)
    X = pd.DataFrame(all_embeddings.numpy())
    return X

# Get features for all sequences in the dataset (X)
X = compute_embeddings(df, 'sequence', checkpoint, 'mean')

# Combine the columns
cols = [n for n in df.columns if n != seqcol]
features = pd.concat([df[seqcol], X, df[cols]],axis=1)
print(features.head(2))


device: cpu


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                                            sequence         0         1  \
0  SKGEELFTGVVPILVELDGEVNGHKFSVSGEGEGDATYGKLTLKFI... -0.060974  0.028259   
1  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI... -0.049561  0.031830   

          2         3         4         5         6         7         8  ...  \
0  0.204102  0.161255  0.226074 -0.047546  0.001337 -0.094971 -0.064819  ...   
1  0.196167  0.151001  0.227783 -0.049774 -0.007648 -0.097412 -0.060760  ...   

        312       313       314       315       316       317       318  \
0 -0.122559  0.094543  0.136230 -0.171143 -0.141479  0.116638  0.022171   
1 -0.113037  0.093323  0.136597 -0.165649 -0.146851  0.100525  0.024429   

        319     label  split  
0  0.034637  3.586350  train  
1  0.030014  3.612519  train  

[2 rows x 323 columns]


**Step 3: Data Preprocessing and Normalization**
<br>Once the features are extracted, the dataset is divided into training and test sets to ensure that the model generalizes well to unseen data. For small datasets, k-fold cross-validation can be used to get more reliable performance estimates (used in this study).

In [7]:
# Define train set for training
train = features[features['split']=='train']
X_train = train.loc[:, seqcol:ycol].iloc[:, 1:-1]
y_train = train[ycol]

# Define test set for evaluation
test = features[features['split']=='test']
X_test = test.loc[:, seqcol:ycol].iloc[:, 1:-1]
y_test = test[ycol]

Define scoring metric to use for evaluation

In [8]:
# In this study, we will use spearman's correlation as the metric for evaluation
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import root_mean_squared_error

def spearmanr_metric(y_true, y_pred):
    r = spearmanr(a=y_true, b=y_pred)
    return r[0]
    
spearmanr_metric.__name__="Spearman"

def pearsonr_metric(y_true, y_pred):
    r = pearsonr(y_true, y_pred)
    return r[0] 

def set_scoring(metric):
    if metric == 'mse':
        return 'neg_root_mean_squared_error'
    if metric == 'acc':
        return 'accuracy'
    elif metric == 'spr':
        return make_scorer(spearmanr_metric)
    elif metric == 'per':
        return make_scorer(pearsonr_metric)
    else:
        print('wrong metric', metric)
        exit()
        
def evaluate_test(y_test, y_pred, metric):
    if metric == 'mse':
        return root_mean_squared_error(y_test, y_pred)
    if metric == 'acc':
        return accuracy_score(y_test, y_pred)
    elif metric == 'spr':
        return spearmanr(y_test, y_pred)[0]
    elif metric == 'per':
        return pearsonr(y_test, y_pred)[0]

**Step 4: Model Selection: Use lazypredict to quickly compare different models on a regression task**
<br> The first step in supervised learning is to compare and select a model for the dataset. 

In this step, the `LazyPredict` Python package is used to identify best model architecture that is suitable for the features. This Python package is designed to quickly try multiple machine learning models with minimal code (using default hyperparameters), making it easier for users to compare the performance of various algorithms without needing to manually implement each model. The package use its own data normalization steps, thus data pre-processing is not needed in this case.

<br>Why lazypredict?
* **Quick Prototyping:** To quickly get a sense of which machine learning models are working best for the dataset before diving into hyperparameter tuning and feature engineering.
* **Model Comparison:** To get potential model candidates that might work best for a given problem/dataset, without having to implement them manually.
* **Exploratory Data Analysis:** To quickly assess the performance of different models to gain insights into how well they fit the data.

Several other AutoML tools are available: AutoGluon, PyCaret, and MLJAR

In [9]:
import sklearn
print(sklearn.__version__)

1.4.2


In [10]:
# Incompatibilities between lazypredict and sklearn might cause errors
from scipy.stats import spearmanr
from sklearn.metrics import make_scorer
from lazypredict.Supervised import LazyRegressor

# Create a LazyRegressor object
rgs = LazyRegressor(verbose=1, ignore_warnings=False, custom_metric=spearmanr_metric)

# Fit the models and get a comparison
models, predictions = rgs.fit(X_train, X_test, y_train, y_test)

# Sort results and show only the selected metric
sorted_models = models.sort_values(by='Spearman', ascending=False)

# Show model comparison results
print(sorted_models[['Spearman','Time Taken']])

  0%|          | 0/42 [00:00<?, ?it/s]

{'Model': 'AdaBoostRegressor', 'R-Squared': -0.18156759543190026, 'Adjusted R-Squared': -0.3510438977798196, 'RMSE': 1.0708033102675003, 'Time taken': 4.5498528480529785, 'Spearman': 0.3377907657529483}
{'Model': 'BaggingRegressor', 'R-Squared': -0.11180725169241623, 'Adjusted R-Squared': -0.2712775881072853, 'RMSE': 1.0387120510972465, 'Time taken': 5.726940870285034, 'Spearman': 0.32418346920131774}
{'Model': 'BayesianRidge', 'R-Squared': -0.013186531737191531, 'Adjusted R-Squared': -0.15851135923871595, 'RMSE': 0.9915739829720861, 'Time taken': 0.9075679779052734, 'Spearman': 0.5484985844661413}
{'Model': 'DecisionTreeRegressor', 'R-Squared': -1.0984120308624115, 'Adjusted R-Squared': -1.3993944826221476, 'RMSE': 1.427006012306691, 'Time taken': 1.0001018047332764, 'Spearman': 0.11553341131049104}
{'Model': 'DummyRegressor', 'R-Squared': -1.1838611208002936, 'Adjusted R-Squared': -1.4970998292969737, 'RMSE': 1.4557705433826131, 'Time taken': 0.03318977355957031, 'Spearman': nan}
{'M

In [14]:
# Print top 5 models for evaluation
print(sorted_models.head(5))

                Adjusted R-Squared  R-Squared  RMSE  Time Taken  Spearman
Model                                                                    
RidgeCV                      -0.17      -0.02  1.00        0.46      0.55
BayesianRidge                -0.16      -0.01  0.99        0.63      0.55
HuberRegressor               -0.32      -0.15  1.06        0.40      0.55
LassoLarsCV                  -0.19      -0.04  1.00        0.85      0.55
ElasticNetCV                 -0.19      -0.04  1.00        8.36      0.55


In [ ]:
# Try autogluon (much better compatibility with latest sklearn)  [NOT TESTED DUE TO COMPATIBILITY ISSUE]  
# It might take some time, but it uses lesser models than lazypredict and pycaret
from autogluon.tabular import TabularPredictor
train = pd.concat([X_train,y_train], axis=1)
test = pd.concat([X_test,y_test], axis=1)

predictor = TabularPredictor(label=ycol, verbosity=1, eval_metric="spearmanr", problem_type="regression").fit(train)
y_pred = predictor.predict(test.drop(columns=[ycol]))
predictor.evaluate(test)
predictor.leaderboard(test)


In [ ]:
# Try pycaret  [NOT TESTED DUE TO COMPATIBILITY ISSUE] 
from pycaret.regression import *
s = setup(train, target = ycol, session_id = 123, verbose=1)

# model training and selection
best = compare_models()

# evaluate trained model
evaluate_model(best)

# predict on hold-out/test set
predict_model(best, data=test)

In [ ]:
# Try MLJAR  [NOT TESTED DUE TO COMPATIBILITY ISSUE] 
from supervised.automl import AutoML
automl = AutoML()
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)

**Step 5: Model Construction and Hyperparameter Tuning: Use Nested K-fold Cross Validation to Estimate Model Performance**
<br>Data scaling and normalization are necessary to ensure that all features contribute equally to the model.

In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit and transform the train set 
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test set (not fit to ensure the set remains unseen)
X_test_scaled = scaler.transform(X_test)

Hyperparameter optimization (HPO) is the process of finding the best set of hyperparameters for a machine learning model to maximize its performance on a validation (test) set.

**(A) Nested cross-validation for small dataset**
When training data are limited, double cross-validation (nested cross-validation) is the standard approach for evaluating and comparing machine learning models with tuned hyperparameters. Nested cross-validation is particularly important when a fixed, untouched test set is not available. Without nested cross-validation, the same data may be used both for hyperparameter selection and performance evaluation, leading to optimistically biased estimates and increased risk of overfitting.

* **Inner Cross-Validation (Hyperparameter Tuning):**
  - Split the data into N_SPLITS folds
  - Evaluate the model's performance using each fold as a validation set while training on the remaining folds
  - Estimate the best set of hyperparameters for the model
* **Outer Cross-Validation (Model Evaluation):**
  - Assess the model's generalization performance
  - Ensures that the model is tested on different, unseen data splits

**(B) Standard Inner Cross-validation**
<br> K-fold cross-validation can be used to estimate model performance by producing multiple performance estimates across different data splits, which is particularly suitable for medium to large datasets.

*Hyperparameter tuning*
<br>Here, Support Vector Regressor (SVR) will be used for further training and evaluation. To find the optimal hyperparameters for the model, several HPO algorithms will be tested.

*Why use K-fold instead of a single train-test split?*
<br>A single train-test split can produce a noisy or biased estimate of model performance, particularly if the test set is small or not representative of the population. K-fold cross-validation mitigates this issue by evaluating the model across multiple folds of the data, providing a more robust and stable estimate of generalization performance.

*Example on how to use K-fold with GridSearchCV*
```
cv = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
gsmodel = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring=scoring, verbose=1)
gsmodel.fit(X, y)
print(gsmodel.best_params_)
print(gsmodel.best_score_)
```

To allow proper comparison on different approaches that use train/test datasets (such as protein language model finetuning later on), hyperparameter tuning and model training are performed on the train set and evaluated on the test set. In the original study, several random seeds were used to obtain average performance scores of the model.

**Hyperparameter Tuning:**
<br> Hyperparameter optimization can be performed using a range of search strategies that differ in efficiency and scalability. GridSearchCV performs an exhaustive evaluation of all parameter combinations in a predefined grid and is straightforward to implement but becomes computationally prohibitive as the search space grows. RandomizedSearchCV improves efficiency by sampling hyperparameter configurations at random, often achieving comparable performance with substantially fewer evaluations. More advanced methods, such as Optuna (including OptunaSearchCV), Hyperopt, and scikit-optimize (skopt), employ Bayesian or adaptive optimization strategies that leverage information from previous trials to guide subsequent searches toward promising regions of the parameter space. These methods are generally more sample-efficient and better suited for large or continuous search spaces, though they introduce additional complexity and dependencies. Overall, grid and random search provide simple and reproducible baselines, while Bayesian optimization frameworks offer improved efficiency for computationally expensive models.


In [36]:
# Define functions for HPO algorithms
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
import optuna
from optuna.distributions import FloatDistribution, IntDistribution, CategoricalDistribution
from optuna_integration.sklearn import OptunaSearchCV
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK 
from skopt import BayesSearchCV
from skopt.space import Real
import numpy as np

# set verbose=1 to show errors, warning and std output (e.g. trial log)
# Hide warnings
import warnings
warnings.filterwarnings('ignore')

def default(model, seed, X_train, X_test, y_train, y_test, metric):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    val = evaluate_test(y_test, y_pred, metric)
    return val, model
    
def train(model, params, seed, X_train, X_test, y_train, y_test, metric):
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    val = evaluate_test(y_test, y_pred, metric)
    return val, model
     
def grid_search(model, param_grid, seed, X_train, X_test, y_train, y_test, metric):
    scoring=set_scoring(metric)
    reg = model
    optim = GridSearchCV(estimator=model, param_grid=param_grid, 
                         cv=5, scoring=scoring, verbose=0)
    optim.fit(X_train, y_train)
    score, est = train(model, optim.best_params_, seed, X_train, X_test, y_train, y_test, metric)
    return (score, est, optim.best_params_)

def random_search(model, param_grid, seed, X_train, X_test, y_train, y_test, metric):
    scoring=set_scoring(metric)
    reg = model
    optim = RandomizedSearchCV(estimator=reg, param_distributions=param_grid, 
                               cv=5, scoring=scoring, verbose=0)
    optim.fit(X_train, y_train)
    score, est = train(model, optim.best_params_, seed, X_train, X_test, y_train, y_test, metric)
    return (score, est, optim.best_params_)

def optuna_search(model, param_grid, seed, X_train, X_test, y_train, y_test, n_trials, metric):
    scoring=set_scoring(metric)
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    # set up param_distributions for optuna
    param_distributions = {}
    for name, values in param_grid.items():
        if len(values) == 2:
            if isinstance(values[0], str):
                param_distributions[name] = CategoricalDistribution(values)
            elif all(isinstance(v, int) for v in values):
                param_distributions[name] = IntDistribution(low=values[0], high=values[1], log=True)
            else:
                param_distributions[name] = FloatDistribution(low=values[0],high=values[1],log=True)
        elif len(values) > 2:
            param_distributions[name] = CategoricalDistribution(values)
        elif len(values) == 1:
            print("Error. Param ranges or list of values was not specified.")
            exit()
        else:
            print("Error. Something wrong with the param grid.")
            exit()
            
    # set up training 
    reg = model
    optim = optuna.integration.OptunaSearchCV(
        reg, param_distributions, n_trials=n_trials, random_state=seed, scoring=scoring, verbose=0
    )
    optim.fit(X_train, y_train)
    score, est = train(model, optim.best_params_, seed, X_train, X_test, y_train, y_test, metric)
    return (score, est, optim.best_params_)

def hyperopt_search(model, param_grid, seed, X_train, X_test, y_train, y_test, n_trials, metric):
    scoring=set_scoring(metric)
    # set up param_distributions for optuna
    param_distributions = {}
    for name, values in param_grid.items():
        if len(values) == 2:
            if isinstance(values[0], str):
                param_distributions[name] = hp.choice(name, values)
            elif all(isinstance(v, int) for v in values):
                #param_distributions[name] = hp.quniform(name, values[0], values[1], q=1)
                param_distributions[name] = hp.choice(name, np.arange(values[0], values[1], dtype=int))
            else:
                param_distributions[name] = hp.loguniform(
                    name,
                    np.log(values[0]),
                    np.log(values[1])
                )
        elif len(values) > 2:
            param_distributions[name] = hp.choice(name, values)
        elif len(values) == 1:
            print("Error. Param ranges or list of values was not specified.")
            exit()
        else:
            print("Error. Something wrong with the param grid.")
            exit()
    # set up training
    reg = model
    def objective(params):
        # Create model with sampled params
        reg.set_params(**params)
        # Cross-validation score (Hyperopt minimizes, so we negate scoring)
        cv_scores = cross_val_score( model, X_train, y_train, scoring=scoring, cv=5, verbose=0)
        loss = -np.mean(cv_scores)
        return { 'loss': loss, 'status': STATUS_OK, 'params': params }
    trials = Trials()
    # Run TPE optimization
    optim = fmin(fn=objective, space=param_distributions, algo=tpe.suggest, 
                 max_evals=n_trials, rstate=np.random.default_rng(seed), trials=trials, verbose=0 )
    score, est = train(model, optim, seed, X_train, X_test, y_train, y_test, metric)
    return (score, est, optim)

def skopt_search(model, param_grid, seed, X_train, X_test, y_train, y_test, n_trials, metric):
    scoring=set_scoring(metric)
    # set up param_distributions for optuna
    param_distributions = {}
    for name, values in param_grid.items():
        if len(values) == 2:
            if isinstance(values[0], str):
                param_distributions[name] = Categorical(values)
            elif all(isinstance(v, int) for v in values):
                param_distributions[name] = Integer(
                    values[0],
                    values[1]
                )
            else:
                param_distributions[name] = Real(
                    values[0],
                    values[1],
                    prior='log-uniform'
                )
        elif len(values) > 2:
            param_distributions[name] = Categorical(values)
        elif len(values) == 1:
            print("Error. Param ranges or list of values was not specified.")
            exit()
        else:
            print("Error. Something wrong with the param grid.")
            exit()
     # set up training
    reg = model
    optim = BayesSearchCV(
        estimator=reg,
        search_spaces=param_distributions,
        n_iter=n_trials,
        random_state=seed,
        scoring=scoring,
        cv=5,                 # same as OptunaSearchCV default
        verbose=0,
        optimizer_kwargs={"base_estimator": "GP"}  # Gaussian Process BO
    )
    optim.fit(X_train, y_train)
    score, est = train(model, optim.best_params_, seed, X_train, X_test, y_train, y_test, metric)
    return (score, est, optim.best_params_)
    

In [37]:
from sklearn.svm import SVR
from sklearn.metrics import make_scorer
#from sklearnex import patch_sklearn
#patch_sklearn()

# Define model
model = SVR()
metric = 'spr'
n_trials=5
param_grid = { 'gamma': [1e-5, 0.5], 'C':[0.01, 30.0], 'epsilon':[0.001, 2.0] }

default, d_estimator = default(model, seed, X_train_scaled, X_test_scaled, y_train, y_test, metric)
print(f'default: {default:.4f}')

gridsearch, g_estimator, g_params = grid_search(model, param_grid, seed, X_train_scaled, X_test_scaled, y_train, y_test, metric)
print(f'gridsearchCV: {gridsearch:.4f}')

randomsearch, r_estimator, r_params = random_search(model, param_grid, seed, X_train_scaled, X_test_scaled, y_train, y_test, metric)
print(f'randomsearchCV: {randomsearch:.4f}')

optuna, o_estimator, o_params = optuna_search(model, param_grid, seed, X_train_scaled, X_test_scaled, y_train, y_test, n_trials, metric)
print(f'optuna: {optuna:.4f}')

hyperopt, h_estimator, h_params = hyperopt_search(model, param_grid, seed, X_train_scaled, X_test_scaled, y_train, y_test, n_trials, metric)
print(f'hyperopt: {hyperopt:.4f}')

skopt, s_estimator, s_params = skopt_search(model, param_grid, seed, X_train_scaled, X_test_scaled, y_train, y_test, n_trials, metric)
print(f'skopt: {skopt:.4f}')


default: 0.4947
gridsearchCV: 0.4827
randomsearchCV: 0.4827


[W 2025-12-24 20:07:40,508] Trial 3 failed with parameters: {'gamma': 0.0003139312453909013, 'C': 0.16358767917148717, 'epsilon': 1.9886440867814372} because of the following error: The value nan is not acceptable.
[W 2025-12-24 20:07:40,508] Trial 3 failed with value nan.


optuna: 0.5365
hyperopt: 0.5459
skopt: 0.4990


In [39]:
# Try three different model architecture
# Note that the param_grid requires a range for float or integers (lower_value, upper_value)
from sklearn.svm import SVR
SVR_param_grid = { 'gamma': [0.1, 0.2], 'C':[20, 30], 'epsilon':[0.1, 1.5] }
SVR_hyperopt, SVR_estimator, SVR_params = hyperopt_search(SVR(), SVR_param_grid, seed, X_train_scaled, X_test_scaled, y_train, y_test, n_trials, metric)
print(f'SVR: {SVR_hyperopt:.4f}, {SVR_params}')

from sklearn.linear_model import Lasso
Lasso_param_grid = { 'alpha': [1e-3, 5e-3] }
Lasso_hyperopt, Lasso_estimator, Lasso_params = hyperopt_search(Lasso(), Lasso_param_grid, seed, X_train_scaled, X_test_scaled, y_train, y_test, n_trials, metric)
print(f'Lasso: {Lasso_hyperopt:.4f}, {Lasso_params}')

from sklearn.ensemble import ExtraTreesRegressor
ETR_param_grid = { 'n_estimators': [10, 50] }
ETR_hyperopt, ETR_estimator, ETR_params = hyperopt_search(ExtraTreesRegressor(random_state=42), ETR_param_grid, seed, X_train_scaled, X_test_scaled, y_train, y_test, n_trials, metric)
print(f'ETR: {ETR_hyperopt:.4f}, {ETR_params}')


SVR: 0.3080, {'C': 9, 'epsilon': 0.5192261094259663, 'gamma': 0.12315040349850609}
Lasso: 0.5432, {'alpha': 0.0012668423699634733}
ETR: 0.3844, {'n_estimators': 33}


Here, train set was used for hyperparameter tuning to obtain unbiased performance estimate using k-fold cross-validation (CV). Instead of using the best estimator output from HPO, the model is re-built with best parameters from HP tuning, re-trained with training set and evaluated on unseen test set. By using the train set for model construction and hyperparameter tuning, the test set remains truly unseen throughout the entire model selection process, ensuring an unbiased evaluation.

**Step 8: Model Prediction**
<br>After evaluating several models based on their performance metrics, the best model can be saved for prediction on unseen or real data. For example, the model can be used to predict the fluorescent intensity of variants, given the amino acid features as the input.

In [40]:
# Define a function to generate all single-residue mutants
def single_residue_mutants(sequence, amino_acids=None):
    if amino_acids is None:
        amino_acids = list("ACDEFGHIKLMNPQRSTVWY")
    mutants = []
    for i, wt in enumerate(sequence):
        for aa in amino_acids:
            if aa != wt:
                mutant = sequence[:i] + aa + sequence[i+1:]
                label = f"{wt}{i+1}{aa}"
                mutants.append([label]+[mutant])
    mutants = pd.DataFrame(mutants)
    mutants.columns = ['id','mutant']
    return mutants

# Generate the list of mutants for prediction
sequence = "SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"
mutants = single_residue_mutants(sequence)
print(mutants.head())
print(len(mutants))

    id                                             mutant
0  S1A  AKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
1  S1C  CKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
2  S1D  DKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
3  S1E  EKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
4  S1F  FKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
4503


**Perform prediction on the prediction list**

In [41]:
# Compute amino acid features for 20 sequences in the dataset
print(len(sequence))

#X_df = mutants.head(100)
X_pred = compute_embeddings(mutants, 'mutant', checkpoint, 'mean')
X_pred.head()

237


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,0,1,2,3,4,5,6,7,8,9,...,310,311,312,313,314,315,316,317,318,319
0,-0.05,0.03,0.20,0.16,0.24,-0.03,0.00,-0.09,-0.05,-0.18,...,0.12,0.03,-0.13,0.09,0.14,-0.19,-0.14,0.12,0.02,0.03
1,-0.06,0.03,0.20,0.16,0.22,-0.04,0.00,-0.10,-0.05,-0.18,...,0.12,0.03,-0.11,0.08,0.13,-0.17,-0.16,0.10,0.04,0.03
2,-0.05,0.03,0.20,0.16,0.24,-0.04,0.01,-0.09,-0.05,-0.19,...,0.12,0.03,-0.13,0.09,0.14,-0.18,-0.14,0.11,0.02,0.02
3,-0.05,0.03,0.20,0.17,0.24,-0.04,0.02,-0.09,-0.05,-0.19,...,0.13,0.03,-0.13,0.09,0.14,-0.17,-0.14,0.10,0.02,0.03
4,-0.06,0.03,0.20,0.15,0.23,-0.04,0.01,-0.09,-0.05,-0.18,...,0.13,0.03,-0.12,0.09,0.14,-0.17,-0.14,0.11,0.02,0.03


**Make prediction**

In [43]:
# Make predictions
X_pred_scaled = scaler.transform(X_pred)

y_pred = h_estimator.predict(X_pred_scaled)
results = pd.concat([mutants[['id']], pd.DataFrame(y_pred, columns=['pred_score'])], axis=1)
results = results.sort_values(by='pred_score', ascending=False)
results.head()

,id,pred_score
2266,N120G,4.46
263,L14V,4.39
2353,L124V,4.32
2192,D116K,4.28
3360,L177V,4.28


**Evaluation of different pooling strategies**

In [44]:

def pool_workflow(pool, df, param_grid, seed, n_trials, metric):
    X = compute_embeddings(df, 'sequence', checkpoint, pool)
    cols = [n for n in df.columns if n != seqcol]
    embed = pd.concat([df[seqcol], X, df[cols]],axis=1)

    train_df = embed[embed['split']=='train']
    Xtrain = train_df.loc[:, seqcol:ycol].iloc[:, 1:-1]
    ytrain = train_df[ycol]
    
    # Define test set for evaluation
    test_df = embed[embed['split']=='test']
    Xtest = test_df.loc[:, seqcol:ycol].iloc[:, 1:-1]
    ytest = test_df[ycol]

    # Feature scaling
    Scaler = StandardScaler() 
    XtrainScaled = Scaler.fit_transform(Xtrain)
    XtestScaled = Scaler.transform(Xtest)

    # Optimize hyperparameters
    score, estimator, params = hyperopt_search(SVR(), param_grid, seed, XtrainScaled, XtestScaled, ytrain, ytest, n_trials, metric)
    return score

for pool in ['mean', 'max', 'sum']:
    score = pool_workflow(pool, df, param_grid, 42, 5, 'spr')
    print(f'{pool}: {score:.4f}')

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mean: 0.5459


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


max: 0.5598


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


sum: 0.5457


**References:** <br>
* [Fluorescence TAPE benchmark dataset](https://github.com/songlab-cal/tape)
* [ESM-2 models](https://github.com/facebookresearch/esm)
* [Protein Language Model](https://huggingface.co/models?other=protein+language+model)
* [scikit-learn.org](https://scikit-learn.org/1.5/auto_examples/model_selection/plot_nested_cross_validation_iris.html)
* [LazyPredict](https://lazypredict.readthedocs.io/en/latest/)
* [PyCaret](https://github.com/pycaret/pycaret)
* [AutoGluon](https://auto.gluon.ai/stable/tutorials/tabular/tabular-quick-start.html)
* [MLJAR](https://supervised.mljar.com)
* [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
* [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
* [Optuna](https://optuna.readthedocs.io/en/v2.0.0/reference/generated/optuna.integration.OptunaSearchCV.html)
* [Hyperopt](https://github.com/hyperopt/hyperopt)
* [Skopt](https://scikit-optimize.github.io/stable/)
* [Model selection: Nested Cross-Validation for Machine Learning with Python](https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/)
* [Model selection: Training-validation-test split and cross-validation done right](https://machinelearningmastery.com/training-validation-test-split-and-cross-validation-done-right/)
* [Model construction: How to Train a Final Machine Learning Model](https://machinelearningmastery.com/train-final-machine-learning-model/)
* [Model training: Embrace Randomness in Machine Learning](https://machinelearningmastery.com/randomness-in-machine-learning/)
